# Debug STAC query for HLS S30, L30

In [16]:
from pystac_client import Client
import pandas as pd

def query_stac(bbox, datetime, cloudcover, collection_list=["HLSS30.v2.0","HLSL30.v2.0"]):
    catalog = Client.open("https://cmr.earthdata.nasa.gov/stac/LPCLOUD")
    
    search = catalog.search(
        collections=collection_list,
        bbox=bbox, #[-122.4969372889167, 59.282959601314104, -120.20175133769581, 60.40424594544261],
        datetime=datetime, #['2020-06-01T00:00:00Z','2021-06-15T23:59:59Z'],
        # max_items=80, # for testing
        # query={"eo:cloud_cover":{"lt":20}} #doesn't work
    )
    results = search.get_all_items_as_dict()
    
    filtered_results = []
    for i in results['features']:
        if int(i['properties']['eo:cloud_cover']) <= cloudcover:
            filtered_results.append(i)
    
    results['features'] = filtered_results
    
    return results


# Manual EarthData search    

In [41]:
HLS_product_version = 2.0
HLS_product_list = [f"HLSL30.v{HLS_product_version}", f"HLSS30.v{HLS_product_version}"]
HLS_product_list = [f"HLSS30.v{HLS_product_version}"]

STARTDATE = '2021-01-01'
ENDDATE = '2021-02-15'

print(f'\nConducting HLS search now... {HLS_product_list}')

manual_link = f'https://search.earthdata.nasa.gov/search/granules?p=C2021957295-LPCLOUD&pg[0][v]=f&pg[0][qt]={STARTDATE}T00%3A00%3A00.000Z%2C{ENDDATE}T23%3A59%3A59.999Z&pg[0][gsk]=-start_date&q=HLSS30&tl=1676070115.136!3!!&lat=33.041963238815576&long=-84.65625000000001'

results = query_stac([-122.45, 59.28, -120.20, 60.40], [f'{STARTDATE}T00:00:00Z',f'{ENDDATE}T23:59:59Z'], 20, collection_list = HLS_product_list)


Conducting HLS search now... ['HLSS30.v2.0']


In [42]:
pd.DataFrame(results).features.to_list()[0]

{'type': 'Feature',
 'id': 'HLS.S30.T11VLH.2021043T192519.v2.0',
 'stac_version': '1.0.0',
 'stac_extensions': ['https://stac-extensions.github.io/eo/v1.0.0/schema.json'],
 'collection': 'HLSS30.v2.0',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-120.6196282, 60.2993491],
    [-118.6339723, 60.3386088],
    [-118.6850101, 61.323456],
    [-120.7302681, 61.2831872],
    [-120.6196282, 60.2993491]]]},
 'bbox': [-120.730268, 60.299349, -118.633972, 61.323456],
 'links': [{'rel': 'self',
   'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSS30.v2.0/items/HLS.S30.T11VLH.2021043T192519.v2.0'},
  {'rel': 'parent',
   'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSS30.v2.0'},
  {'rel': 'collection',
   'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSS30.v2.0'},
  {'rel': 'root', 'href': 'https://cmr.earthdata.nasa.gov/stac/'},
  {'rel': 'provider', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD'},
  {'rel': 'via',
   'hr